Majority vote from team's CV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob

from tqdm import tqdm
import seaborn as sns
from collections import Counter

import cv2

In [2]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [3]:
from collections import Counter
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [4]:
from collections import defaultdict

In [5]:
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [7]:
#exps = !ls ../input/whale-21-3-2022/ | grep pbz2 | grep $fold
#exps

['exp16v6AugMix1536_fold4.pbz2',
 'exp16v6AugmixB3_fold4.pbz2',
 'exp16v6AugmixB4Step2_fold4.pbz2',
 'exp16v6AugmixB4_fold4_remove_background.pbz2',
 'exp16v6Augmix_fold4.pbz2']

In [9]:
FOLD=4

In [10]:
exps_all = [
    [
         'exp16v6Augmix_fold0.pbz2',
         'exp16v4sampler_fold0.pbz2',
         'exp16v6AugMixNew4_fold0.pbz2',
         'exp16v6AugMixNew4_fold0_tta.pbz2',
         'exp16v6AugmixB3_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0_remove_background.pbz2',
         'exp16v6AugmixB4_fold0.pbz2',
         'exp16v6AugmixB4_fold0_remove_background.pbz2',
         'exp16v6AugmixFullB4_fold0.pbz2',
         'exp16v6sampler_fold0.pbz2',
         'exp21b6_fold0.pbz2',
         'exp21b6_fold0_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold1.pbz2',
         'exp16v6AugMixNew4_fold1.pbz2',
         'exp16v6AugMixNew4_fold1_tta.pbz2',
         'exp16v6AugmixB3_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1_remove_background.pbz2',
         'exp16v6AugmixB4_fold1.pbz2',
         'exp21b6_fold1_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold2.pbz2',
         'exp16v6AugMixNew4_fold2.pbz2',
         'exp16v6AugMixNew4_fold2_tta.pbz2',
         'exp16v6AugmixB3_fold2.pbz2',
         'exp16v6AugmixB4Full_fold2.pbz2',
         'exp16v6AugmixB4Step2_fold2.pbz2',
         'exp16v6AugmixB4_fold2.pbz2',
         'exp16v6AugmixB4_fold2_remove_background.pbz2',
         'exp21b6_fold2_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold3.pbz2',
         'exp16v6AugMix1536_fold3.pbz2',
         'exp16v6AugmixB3_fold3.pbz2',
         'exp16v6AugmixB4Full_fold3.pbz2',
         'exp16v6AugmixB4Step2_fold3.pbz2',
         'exp16v6AugmixB4_fold3_remove_background.pbz2',
         'exp21b6_fold3_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold4.pbz2',
         'exp16v6AugMix1536_fold4.pbz2',
         'exp16v6AugmixB3_fold4.pbz2',
         'exp16v6AugmixB4Step2_fold4.pbz2',
         'exp16v6AugmixB4_fold4_remove_background.pbz2',
    ]
]

In [11]:
exps = exps_all[FOLD]

In [12]:
fold2 = pd.read_csv("/kaggle/happywhale/input/whale-21-3-2022/fold2.csv")
fold2.shape

(51033, 8)

In [13]:
valid = fold2.query("fold == @FOLD").reset_index()
train = fold2.query("fold != @FOLD").reset_index()

In [14]:
valid_ind = (fold2.fold == FOLD).values
train_ind = (fold2.fold != FOLD).values

In [15]:
train_inddividual_id_lis = train["individual_id"].values
valid_inddividual_id_lis = valid["individual_id"].values

def get_result_each(files):
    DIRNAME ="../input/whale-21-3-2022/"
      
    ret = []
        
    for fname in files:
        print(fname)
        data = decompress_pickle(DIRNAME + fname  )
        
        train_emb = data["emb_val"]#[new_index,:]
        
        train_emb=normalize(train_emb)
        
        valid_preds = train_emb[valid_ind]
        train_preds = train_emb[train_ind]
        
        
        distances = pairwise_distances(valid_preds,train_preds)
                
        K=1000
        predicted_positions = np.argpartition(distances, K, axis=1)[:, :K]

        all_pred_lis = []
        for i in tqdm(range(predicted_positions.shape[0])):
            nearest = [(train_inddividual_id_lis[j], distances[i, j]) for j in predicted_positions[i]]
            nearest.sort(key=lambda x: x[1])

            prediction = [index_id for index_id, d in nearest]

            done = set()
            pred_list = []
            for pred_id in prediction:
                if pred_id in done:
                    continue
                done.add(pred_id)
                pred_list.append(pred_id)
                if len(pred_list)==20:
                    break

            all_pred_lis.append(pred_list)
        
        ret.append(all_pred_lis)
        
    return ret
    

### get result for each model

In [16]:
ret = get_result_each(exps)

exp16v6Augmix_fold4.pbz2


100%|██████████| 8421/8421 [00:07<00:00, 1124.55it/s]


exp16v6AugMix1536_fold4.pbz2


100%|██████████| 8421/8421 [00:08<00:00, 1046.63it/s]


exp16v6AugmixB3_fold4.pbz2


100%|██████████| 8421/8421 [00:07<00:00, 1120.59it/s]


exp16v6AugmixB4Step2_fold4.pbz2


100%|██████████| 8421/8421 [00:07<00:00, 1115.51it/s]


exp16v6AugmixB4_fold4_remove_background.pbz2


100%|██████████| 8421/8421 [00:07<00:00, 1117.60it/s]


In [17]:
len(ret),len(ret[0]),len(ret[0][0])

(5, 8421, 20)

In [18]:
def get_score(weights):
    
    #train_emb
    vote_result = defaultdict(lambda:defaultdict(int))
    
    for preds, w in zip(ret, weights):
        for i in range(len(preds)):
            
            for j,p in enumerate(preds[i]):
                vote_result[i][p] += (w / (j+1))
        
        
    score_lis = []
    pred_ids = []

    for i in range(len(ret[0])):
        pred_list = np.array(sorted(vote_result[i].items(), key=lambda item: item[1])[::-1])[:5,0].tolist()

        if valid_inddividual_id_lis[i] in pred_list:
            s = 1/(pred_list.index(valid_inddividual_id_lis[i]) + 1)
            score_lis.append(s)
        else:
            score_lis.append(0)
    
    return np.mean(score_lis)

In [19]:
# un-weighted result
get_score([1] * len(exps))

0.8379942999643747

optimize by optuna

In [21]:
import optuna

In [27]:
def objective_wrapper(exps):
    def objective(trial):

        weights = [1]
        for i in range(len(exps)-1):
            w=trial.suggest_uniform(exps[i+1], 0, 4)
            weights.append(w)
        
        
        score_lis = get_score(weights)
        score = np.mean(score_lis)
        return score
    
    return objective


In [33]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective_wrapper(exps), timeout=300)

[I 2022-04-17 07:05:05,514] A new study created in memory with name: no-name-279432b7-0c1c-4874-8351-46ff31f29c3e
[I 2022-04-17 07:05:06,629] Trial 0 finished with value: 0.8426552665954162 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.3702205109147405, 'exp16v6AugmixB3_fold4.pbz2': 2.8982103314019345, 'exp16v6AugmixB4Step2_fold4.pbz2': 3.178416478749965, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 0.09842233843025916}. Best is trial 0 with value: 0.8426552665954162.
[I 2022-04-17 07:05:07,598] Trial 1 finished with value: 0.8149150932193325 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 0.06397475381760742, 'exp16v6AugmixB3_fold4.pbz2': 3.336295069268839, 'exp16v6AugmixB4Step2_fold4.pbz2': 0.09559487702252545, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 3.8313821782092097}. Best is trial 0 with value: 0.8426552665954162.
[I 2022-04-17 07:05:08,571] Trial 2 finished with value: 0.8427502671891699 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 2.611458442648429, 'exp16v

[I 2022-04-17 07:05:27,995] Trial 22 finished with value: 0.8425187032418952 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.9242350129011974, 'exp16v6AugmixB3_fold4.pbz2': 1.4757253068696086, 'exp16v6AugmixB4Step2_fold4.pbz2': 3.971577713847811, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.762588344864822}. Best is trial 16 with value: 0.8492399952499703.
[I 2022-04-17 07:05:28,972] Trial 23 finished with value: 0.8430926651624906 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 2.8675571789853134, 'exp16v6AugmixB3_fold4.pbz2': 2.2146593733367137, 'exp16v6AugmixB4Step2_fold4.pbz2': 2.5682285276760433, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.095200203522669}. Best is trial 16 with value: 0.8492399952499703.
[I 2022-04-17 07:05:29,934] Trial 24 finished with value: 0.8486779084035942 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.5852460107825945, 'exp16v6AugmixB3_fold4.pbz2': 0.6872213982468249, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.998429245879706, 'exp16v6AugmixB

[I 2022-04-17 07:05:49,344] Trial 44 finished with value: 0.8486205122115347 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.171639551456522, 'exp16v6AugmixB3_fold4.pbz2': 1.4976422073575169, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.6768992060132606, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 0.32805648866090165}. Best is trial 39 with value: 0.8498832284368443.
[I 2022-04-17 07:05:50,306] Trial 45 finished with value: 0.8405771286070539 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.5787592431356545, 'exp16v6AugmixB3_fold4.pbz2': 0.42889339992985454, 'exp16v6AugmixB4Step2_fold4.pbz2': 2.3433274669006177, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 3.6199115940561697}. Best is trial 39 with value: 0.8498832284368443.
[I 2022-04-17 07:05:51,265] Trial 46 finished with value: 0.8500415627597673 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.7770746832956807, 'exp16v6AugmixB3_fold4.pbz2': 1.1597543004107114, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.3952593387580885, 'exp16v6Au

[I 2022-04-17 07:06:10,547] Trial 66 finished with value: 0.848576970272731 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.6946270640749352, 'exp16v6AugmixB3_fold4.pbz2': 0.25817182939490113, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.6250475287966006, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 0.5664451197785341}. Best is trial 46 with value: 0.8500415627597673.
[I 2022-04-17 07:06:11,510] Trial 67 finished with value: 0.8488857222024305 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.4964631965128214, 'exp16v6AugmixB3_fold4.pbz2': 0.555690384442413, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.3457735578892016, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.4072466148976093}. Best is trial 46 with value: 0.8500415627597673.
[I 2022-04-17 07:06:12,472] Trial 68 finished with value: 0.8465779994458298 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.878297974618885, 'exp16v6AugmixB3_fold4.pbz2': 0.029854603317218897, 'exp16v6AugmixB4Step2_fold4.pbz2': 0.730170746351998, 'exp16v6Augm

[I 2022-04-17 07:06:31,742] Trial 88 finished with value: 0.8404346277164233 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 1.5547472870664842, 'exp16v6AugmixB3_fold4.pbz2': 0.9391025992382808, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.655205367778712, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.0852628780195361}. Best is trial 46 with value: 0.8500415627597673.
[I 2022-04-17 07:06:32,700] Trial 89 finished with value: 0.8502137513359458 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.6257893277134863, 'exp16v6AugmixB3_fold4.pbz2': 1.2813683701459224, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.4974392792316293, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.2682582757389622}. Best is trial 89 with value: 0.8502137513359458.
[I 2022-04-17 07:06:33,660] Trial 90 finished with value: 0.8496773938170448 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.8933653903859478, 'exp16v6AugmixB3_fold4.pbz2': 1.2733701444707073, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.8134059584027542, 'exp16v6Augm

...

[I 2022-04-17 07:09:00,504] Trial 242 finished with value: 0.8503661481217591 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.915458163945765, 'exp16v6AugmixB3_fold4.pbz2': 1.9973175817048612, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.2057823439968185, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.0297005754165631}. Best is trial 200 with value: 0.8505462534140839.
[I 2022-04-17 07:09:01,467] Trial 243 finished with value: 0.8503582314056128 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.9291264926794067, 'exp16v6AugmixB3_fold4.pbz2': 2.0177462220060383, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.1907722222409827, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.0345046209218167}. Best is trial 200 with value: 0.8505462534140839.
[I 2022-04-17 07:09:02,466] Trial 244 finished with value: 0.8492558286822626 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.9280344766417223, 'exp16v6AugmixB3_fold4.pbz2': 2.2490487357076603, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.2001290761428924, 'exp16v

[I 2022-04-17 07:09:21,968] Trial 264 finished with value: 0.8505106281914263 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.8277342798561698, 'exp16v6AugmixB3_fold4.pbz2': 1.9421986344963793, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.0510625303611574, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.1845034097613663}. Best is trial 200 with value: 0.8505462534140839.
[I 2022-04-17 07:09:22,946] Trial 265 finished with value: 0.849168744804655 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.7608757516070277, 'exp16v6AugmixB3_fold4.pbz2': 2.231937086536539, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.0992421824484384, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.2032209054081497}. Best is trial 200 with value: 0.8505462534140839.
[I 2022-04-17 07:09:23,910] Trial 266 finished with value: 0.8462692475161303 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 2.5395368002994627, 'exp16v6AugmixB3_fold4.pbz2': 1.8538536930889564, 'exp16v6AugmixB4Step2_fold4.pbz2': 0.7501304781078795, 'exp16v6

[I 2022-04-17 07:09:43,264] Trial 286 finished with value: 0.8492597870403356 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.7720702815076206, 'exp16v6AugmixB3_fold4.pbz2': 2.2957630065509567, 'exp16v6AugmixB4Step2_fold4.pbz2': 0.6998745925156672, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.2452099863754762}. Best is trial 200 with value: 0.8505462534140839.
[I 2022-04-17 07:09:44,285] Trial 287 finished with value: 0.8265764161026007 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 0.5505454040785753, 'exp16v6AugmixB3_fold4.pbz2': 1.9335137764479136, 'exp16v6AugmixB4Step2_fold4.pbz2': 0.9364286350478522, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.2413825085018002}. Best is trial 200 with value: 0.8505462534140839.
[I 2022-04-17 07:09:45,250] Trial 288 finished with value: 0.8484740529628311 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.8809491484482774, 'exp16v6AugmixB3_fold4.pbz2': 2.156678746896956, 'exp16v6AugmixB4Step2_fold4.pbz2': 0.25532249960479025, 'exp16

[I 2022-04-17 07:10:04,692] Trial 308 finished with value: 0.850387919091161 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.9634374592462724, 'exp16v6AugmixB3_fold4.pbz2': 1.8323283984332142, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.2777079333876908, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.123465739147828}. Best is trial 305 with value: 0.8506056287851799.
[I 2022-04-17 07:10:05,696] Trial 309 finished with value: 0.8505779202786685 and parameters: {'exp16v6AugMix1536_fold4.pbz2': 3.996183937203543, 'exp16v6AugmixB3_fold4.pbz2': 1.8193517637126337, 'exp16v6AugmixB4Step2_fold4.pbz2': 1.1785772888370318, 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.2035596932311565}. Best is trial 305 with value: 0.8506056287851799.


CPU times: user 5min, sys: 1.45 s, total: 5min 1s
Wall time: 5min


In [34]:
study.trials_dataframe().sort_values("value")

,number,value,datetime_start,datetime_complete,duration,params_exp16v6AugMix1536_fold4.pbz2,params_exp16v6AugmixB3_fold4.pbz2,params_exp16v6AugmixB4Step2_fold4.pbz2,params_exp16v6AugmixB4_fold4_remove_background.pbz2,state
1,1,0.814915,2022-04-17 07:05:06.630533,2022-04-17 07:05:07.598466,0 days 00:00:00.967933,0.063975,3.336295,0.095595,3.831382,COMPLETE
113,113,0.820827,2022-04-17 07:06:54.901880,2022-04-17 07:06:55.893739,0 days 00:00:00.991859,0.170659,1.190871,1.438718,2.523521,COMPLETE
91,91,0.822652,2022-04-17 07:06:33.661667,2022-04-17 07:06:34.630850,0 days 00:00:00.969183,0.013844,1.298012,1.478919,1.506646,COMPLETE
287,287,0.826576,2022-04-17 07:09:43.265563,2022-04-17 07:09:44.285311,0 days 00:00:01.019748,0.550545,1.933514,0.936429,1.241383,COMPLETE
35,35,0.827895,2022-04-17 07:05:39.679864,2022-04-17 07:05:40.655439,0 days 00:00:00.975575,0.352618,1.330537,0.895245,0.906162,COMPLETE
...,...,...,...,...,...,...,...,...,...,...
264,264,0.850511,2022-04-17 07:09:21.003847,2022-04-17 07:09:21.968314,0 days 00:00:00.964467,3.827734,1.942199,1.051063,1.184503,COMPLETE
209,209,0.850524,2022-04-17 07:08:27.677807,2022-04-17 07:08:28.637280,0 days 00:00:00.959473,3.849565,1.960261,1.120766,1.085934,COMPLETE
200,200,0.850546,2022-04-17 07:08:19.020569,2022-04-17 07:08:19.974932,0 days 00:00:00.954363,3.853196,1.846484,1.130525,1.106594,COMPLETE
309,309,0.850578,2022-04-17 07:10:04.693651,2022-04-17 07:10:05.696422,0 days 00:00:01.002771,3.996184,1.819352,1.178577,1.203560,COMPLETE


In [35]:
study.best_params

{'exp16v6AugMix1536_fold4.pbz2': 3.939976169388798,
 'exp16v6AugmixB3_fold4.pbz2': 1.8657933946604268,
 'exp16v6AugmixB4Step2_fold4.pbz2': 1.1010188935628786,
 'exp16v6AugmixB4_fold4_remove_background.pbz2': 1.1909781255471734}

In [36]:
best_weights = list(study.best_params.values())

In [38]:
# weighted result
get_score([1] + best_weights)

0.8506056287851799

In [39]:
# un-weighted result
get_score([1] * len(exps))

0.8379942999643747